In [1]:
print("hello")

hello


In [4]:
%pwd

'd:\\Python Practice\\Medical-Chatbot'

In [22]:
import os
os.chdir("../")

In [23]:
%pwd


'd:\\Python Practice\\Medical-Chatbot'

In [24]:
os.chdir("medical-chatbot")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'medical-chatbot'

In [25]:
%pwd

'd:\\Python Practice\\Medical-Chatbot'

In [26]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [28]:
import os
import PyPDF2
extracted_data = load_pdf_file(data='data/')

In [30]:
#extracted_data

In [31]:
def test_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [32]:
text_chunks = test_split(extracted_data)
print("length of text chunks:",len(text_chunks))

length of text chunks: 39994


In [33]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [34]:
embeddings = download_hugging_face_embeddings()

In [35]:
query_result = embeddings.embed_query("hello")
print("query result:",query_result)
print("length of query result:",len(query_result))

query result: [-0.0627717599272728, 0.05495879426598549, 0.052164845168590546, 0.0857900083065033, -0.0827488973736763, -0.07457297295331955, 0.06855470687150955, 0.018396394327282906, -0.08201129734516144, -0.03738484904170036, 0.012124893255531788, 0.0035183229483664036, -0.004134293645620346, -0.04378441721200943, 0.021807299926877022, -0.00510272104293108, 0.019546564668416977, -0.0423487089574337, -0.11035966873168945, 0.005424527917057276, -0.055734783411026, 0.02805238962173462, -0.02315872348845005, 0.028481375426054, -0.05370955914258957, -0.052601538598537445, 0.03393925353884697, 0.045388635247945786, 0.023718422278761864, -0.07312081754207611, 0.05477774888277054, 0.0170473363250494, 0.08136039972305298, -0.0028627056162804365, 0.011958062648773193, 0.07355856150388718, -0.09423744678497314, -0.0813620463013649, 0.04001539200544357, 0.0006921913009136915, -0.01339334063231945, -0.05453811213374138, 0.005151401273906231, -0.026139769703149796, 0.0368071012198925, -0.03395958

In [36]:
import pinecone
print(pinecone)

<module 'pinecone' from 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib\\site-packages\\pinecone\\__init__.py'>


In [37]:
import sys
print(sys.path)


['c:\\Users\\drmsi\\.conda\\envs\\medibot\\python310.zip', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\DLLs', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib', 'c:\\Users\\drmsi\\.conda\\envs\\medibot', '', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib\\site-packages', 'd:\\python practice\\medical-chatbot', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib\\site-packages\\win32', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib\\site-packages\\win32\\lib', 'c:\\Users\\drmsi\\.conda\\envs\\medibot\\lib\\site-packages\\Pythonwin']


In [38]:
print(pinecone.__version__)

5.4.2


In [39]:
import os
print(os.path.abspath(pinecone.__file__))

c:\Users\drmsi\.conda\envs\medibot\lib\site-packages\pinecone\__init__.py


In [47]:
from dotenv import load_dotenv
import os 
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec



pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [43]:
#embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

KeyboardInterrupt: 

In [46]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import  ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following information to answer the questions."
    "If you don't know the answer, say that you do not have enough information."
    "Use three sentences to answer the question and keep them concise."
    "\n\n"
    "{context:} "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [49]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

NameError: name 'docsearch' is not defined

In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [54]:
from dotenv import load_dotenv
import os

load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
print("DEEPSEEK_API_KEY:", DEEPSEEK_API_KEY)


DEEPSEEK_API_KEY: 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


In [53]:
import pinecone
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key="pcsk_2SFuPC_T5eBWFtaHMn3rSJskWmLtk1V5BNEn7Y5FNWk1HJYot4cguwXVC3iAnoC2c4gJgQ")
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=2, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

KeyboardInterrupt: 

In [ ]:
response = rag_chain.invoke("What is the treatment for diabetes?")
print(response)